## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server.

In [3]:
!pip install optuna


In [7]:
from mlflow.optuna.storage import mlflow_optuna_status_map



import mlflow
import os

os.environ["AWS_PROFILE"] = "default" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "13.218.53.74" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://13.218.53.74:5000'


In [9]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-vivek2/artifacts/0', creation_time=1752088259213, experiment_id='0', last_update_time=1752088259213, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/10 01:00:18 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/07/10 01:00:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 01:00:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-vivek2/artifacts/1/dbc840cde70046a887d541cb2cb1c9fd/artifacts'
🏃 View run redolent-midge-986 at: http://13.218.53.74:5000/#/experiments/1/runs/dbc840cde70046a887d541cb2cb1c9fd
🧪 View experiment at: http://13.218.53.74:5000/#/experiments/1


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-vivek2/artifacts/1', creation_time=1752089418627, experiment_id='1', last_update_time=1752089418627, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-vivek2/artifacts/0', creation_time=1752088259213, experiment_id='0', last_update_time=1752088259213, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [12]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [13]:
client.search_registered_models()

[]

In [14]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/07/10 01:01:54 WARNING mlflow.tracking._model_registry.fluent: Run with id dbc840cde70046a887d541cb2cb1c9fd has no artifacts at artifact path 'models', registering model based on models:/m-e259c2c774cd436fa3b61316f4ccefad instead
2025/07/10 01:01:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1752089515379, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1752089515379, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='dbc840cde70046a887d541cb2cb1c9fd', run_link='', source='models:/m-e259c2c774cd436fa3b61316f4ccefad', status='READY', status_message=None, tags={}, user_id='', version='1'>